# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> AC207 Systems Development for Computational Science


## Milestone 1 : Document


**Harvard University**<br/>
**Fall 2021**<br/>
**Project Group #32**: Xuliang Guo.   Van Anh Le.  Hanwen Cui.  Kamran Ahmed.

<hr style="height:2pt">

In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/"
    "themes/static/css/cs109.css"
).text
HTML(styles)

## Part1. Introduction

Almost all machine learning algorithms can be attributed to solving optimization problems during the training process. If the analytical solution exists, we can achieve the final result by simply substituting numerical calculation. However, not all problems are amenable to analytical solutions. Many times we need to use gradient descent, Newton method, quasi-Newton method, and other numerical optimization algorithms to solve such problems. These algorithms largely rely on the first or second derivative, gradient, Jacobian matrix, Hessian matrix, etc.

- Let's define $f(x)=x^{2} + y^{2} $, and we want its derivative respect to x and y.  ie $\frac{\partial f}{\partial x} $ and $\frac{\partial f}{\partial y} $

For programming to solve the derivative of a function, there are currently 4 methods we can choose:  numerical differentiation, manual differentiation, symbolic differentiation, and what we will introduce -automatic differentiation.

> Automatic Differentiation (AD), also known as computational differentiation, is a method that combines the advantages of symbolic differentiation and numerical differentiation that can efficiently and accurately evaluate derivatives of functions that have been expressed as a computer program.


## Part 2. Background


### Numerical Differentiation


According to the original definition of derivative 

$$ \begin{aligned}
&\frac{\partial f}{\partial x}(a, b)=\lim _{\epsilon \rightarrow 0} \frac{f(a+\epsilon, b)-f(a, b)}{\epsilon} \\
&\frac{\partial f}{\partial y}(a, b)=\lim _{\epsilon \rightarrow 0} \frac{f(a, b+\epsilon)-f(a, b)}{\epsilon}
\end{aligned} $$



we will get an approximate derivative value as long as $\epsilon$ takes a very small value, such as 0.0001. In other words, the numerical method only takes three arguments (objective function, the current value, and the precision(0.0001)) to be able to output an approximate derivative for the current value. However,  the numerical differentiation method is the most computationally expensive among the four methods when there are multiple variables.

From the demo python code below, we can see that the Simple_numerical_diff_x function calls f-function twice.
```python
def Simple_numerical_diff_x(f,x,y x_eps):
    return (f(x + x_eps,y) - f(x,y)) / (x_eps)
```
For the one variable scenario above, the Simple_numerical_diff calls f() twice. For an equation with n variables, to compute the gradient of all dimensions, f() will be called at least n+1 times. When dealing with a large neural network, such operations will greatly reduce our efficiency.

To make matters worse, the roundoff error and truncation error caused by the numerical approximation makes it even less suitable for practical application scenarios. Even if we make $\epsilon$ equal to 1e-10, the roundoff error will still exist, but the calculations will slow down even more due to the sharp increase of digits. For the truncation error, it can be reduced not be eliminated by the the following center difference approximation: $f^{\prime}(x)=\lim _{\epsilon \rightarrow 0} \frac{f(x+\epsilon ,y)-f(x-\epsilon ,y)}{2 \epsilon }$. Although the numerical differentiation can not give a precise result, it is often used to check the results of other methods within a given accuracy.


### Manual Differentiation

Another method is we grab a pen and a piece of paper, manually write down the derivate formula based on our calculus knowledge and then write it into python code.

$$ \frac{\partial f(x,y)}{\partial x}=\frac{\partial\left(x^{2}+y^{2}\right )}{\partial x} =2x$$

However, within the manual method, every time we modify the objective function, the stored derivative expression need also be re-write. The method may become very wordy and time-costly when it is applied to more complex functions. So the manual differentiation obviously cannot be applied to automated machine learning algorithms due to the requirements of human intervention.


### Symbolic Differentiation

Symbolic differentiation is a method used to replace the above manual differentiation, using algebraic programming to implement the automatic derivative of some combined functions, such as:

$$\begin{gathered}
\frac{d}{d x}(f(x)+g(x))=\frac{d}{d x} f(x)+\frac{d}{d x} g(x) \\
\frac{d}{d x} f(x) g(x)=\left(\frac{d}{d x} f(x)\right) g(x)+f(x)\left(\frac{d}{d x} g(x)\right) \\
\end{gathered}$$

Symbolic differentiation first expands the objective function to a tree diagram and then calculates and stores the partial derivative of all leaf nodes and stores them in memory. Treat all the stored expression as a new element, the symbolic differentiation iteratively compute the composite derivative of the parents node. After several upward iterations, we will finally obtain the symbolic expression of composited elements and it is easy to obtain the final answer by simply substituting the initial value.

A significant problem with this method is "expression swell". For a complex objective function, the symbolic method will produce a huge calculation graph, which can't be simplified and will lead to an extremely slow calculation through the entire expression trees.

> - Numerical differentiation emphasizes the direct substitution of finite numerical approximation at the beginning; 
> - Symbolic differentiation emphasizes generating an algebraic derivative expression and then substituting the value at the final step;


### Automatic Differentiation

> - Automatic differentiation will apply the symbolic differentiation method to the most basic operators, such as constants, power functions, exponential functions, logarithmic functions, trigonometric functions, etc., Then substitutes the initial value and applies rules of composition derivative, we achieve the final answer of a wide class of functions automatically and precisely. Due to the basic of only applying symbolic differentiation law to basic functions, it can flexibly combine the loop structure and conditional structure of the programming language without worrying about the problem of expression swell.

### AD Forward Mode and Dual numbers
AD computes the derivative of a complex function $f(x,y)$ by breaking it down into a series of intermediate elementary operations. The chain rule allows us to use the derivative of these intermediate functions to compute $\frac{\partial f}{\partial x}$. In the forward mode AD, we work from the inside out, i.e starting with the value of $x$ at which we want to evaluate and ending with the function $f(x,y)$. 

The evaluation trace table and computational graph below demonstrate this process:
* We'll use the function defined in the introduction, $f(x)=x^{2} + y^{2}$, and evaluate $\frac{\partial f}{\partial x} $ and $\frac{\partial f}{\partial y} $ at $(x,y) = (1,1)$.
* The first two traces $x_{0}$ and $x_{1}$ are the starting values $(x,y) = (1,1)$. 
* For each subsequent trace, elementary operations are applied to the previous traces, until we reach the final function in the last trace. The elementary operations applied are represented on the arrows in the computational graph.
* At each trace, both the derivative ($\nabla_{x}$, $\nabla_{y}$) and value of the intermediate operations are computed. We arrived at the derivatives $(\nabla_{x}$, $\nabla_{y}) = (2,2)$ in the last trace.

#####  Evaluation Trace Table
\begin{align}
 f(x,y)=x^{2} + y^{2} 
\end{align}


| trace   | elem op.    | value | elem der.           | $\nabla_{x}$| $\nabla_{y}$ |
| :---:   | :------:    | :---: | :-------:           | :---------: | :----------: |
| $x_{0}$ | $$x$$       | $$1$$ | $$\dot{x}$$ | $$1$$         | $$0$$          |
| $x_{1}$ | $y$         | $1$   | $\dot{y}$   | $0$           | $1$          |
| $v_{1}$ | $x_{1}^{2}$ | $1$   | $2x_{1}\dot{x_{1}}$ | $0 $         | $2$          |
| $v_{2}$ | $x_{0}^{2}$ | $1$   | $2x_{0}\dot{x_{0}}$ | $2$          | $0$          |
| $f$ | $v_{2} + v_{3}$ | $2$   | $\dot{v_1}+\dot{v_2}$ | $2$         | $2$          |


##### Computational Graph - Forward Mode

![alt text](graph1_forward.png "forward_mode")

As we saw from the trace table, both the derivative and value of the intermediate operations are computed at each trace. Dual numbers can be used as a data structure to store and carried these values forward, where each value and derivative pair can be stored as the real part and the dual part of a dual number respectively. The addition and multiplication properties of dual numbers, as well as the fact that its derivative can be computed using the Taylor series, make it a useful data structure for forward mode AD.


### AD Reverse Mode
In reverse mode AD, instead of working from the inside out, we apply the chain rule backwards. To do this, we first need to perform a forward pass where the partial derivative of intermediate variables with respect to its parent are computed. We then performed a reverse pass and apply the chain rule starting from the last intermediate variable using the computed partial derivative. 

##### Computational Graph - Reverse Mode

![alt text](graph2_reverse.png "reverse_mode")



## Part 3. How to Use --- autodiff
Package will be installed from GitHub or PyPI. Users will interact with our package by instantiating `Dual` numbers and constructing user-defined functions which automatically evaluates the function at the specified values and computes the gradient(s).

**Function with single argument**

```python
>>> import autodiff as ad
>>> x = ad.Dual(2) # instatiate Dual object with value 2 (derivative defaults to 1)
>>> f = 7 * (x ** 3) + 3 * x # define and evaluate function
>>> f.val # function output
62
>>> f.der # derivative
array([87.])

```

**Multivariable functions** 

We provide flexible options to support this.

*Option 1*: create multiple Dual numbers via `Dual.from_array`, which constructs proper seed vectors for each variable. i.e. Dual numbers will have value of `X[i]` and zero derivative vector where the `i-th` element has a value of 1.

```python
>>> import autodiff as ad
>>> X = [2, 4]
>>> x, y = ad.Dual.from_array(X) # helper static method
>>> x
Dual(2, array([1., 0.]))
>>> y
Dual(4, array([0., 1.]))
>>> f = 7 * (x ** 3) + 3 * y
>>> f.val
68
>>> f.der
array([84., 3.]) # df/dx, df/dy
```

*Option 2*: user defines seed.
```python
>>> import autodiff as ad
>>> x = ad.Dual(2, der=[1, 0])
>>> y = ad.Dual(4, der=[0, 1])
```

**Elementary operations**

We will allow users to import overloaded elementary functions (sine, cosine, tangent, exponential, log, sqrt) to perform operations on Dual numbers.

```python
>>> import autodiff as ad
>>> x = ad.Dual(0)
>>> ad.cos(x) # overloaded function which operates on `Dual` numbers
Dual(1, array([-0.]))
```

**Potential enhancement** 

Per Fabian's suggestions, we may implement a lazy evaluation method by constructing a computational graph when a user defines a function using node/graph variables and evaluate the function later. i.e.

```python
>>> import autodiff as ad
>>> x, y = ad.graph_variables(2)
>>> f = 7 * (x ** 3) + 3 * y # define function
>>> f([2, 4]) # evaluate function with values for x and y
68
>>> f.grad([2, 4]) # compute all gradient components
array([84., 3.])
```

## Part 4. Software Organization

**Project structure**

```zsh
cs107-FinalProject
├── autodiff # main package where we import necessary objects and functions from `src`
│   ├── src
│   │   ├── extensions # extensions package (TBD)
│   │       └── __init__.py
│   │   └── forward # forward mode package
│   │       ├── __init__.py
│   │       ├── dual.py
│   │       └── operations.py
│   ├── tests # testing functions for all functions and methods in src
│   │   ├── __init__.py
│   │   ├── test_dual.py
│   │   └── test_operations.py
│   └── __init__.py
├── docs
│   ├── README.md # README for usage and tutorials
│   └── milestone1.ipynb
├── .gitignore
├── .travis.yml # TravisCI build configuration
├── LICENSE
├── pyproject.toml # minimum build system requirements 
├── README.md # main README for installation and overview
├── requirements.txt # necessary dependencies
└── setup.cfg # setuptools configuration

```

**Modules and packages**
- *autodiff/*: Main package where we import necessary objects and functions from *extensions* and *forward* packages. This will provide basic functionality to initialize our package (`import autodiff as ad`).


- *autodiff/src/extensions*: Extensions package for our AD algorithm. We are still in the process of deciding what direction(s) we want to take here. Examples include reverse mode implementation, computational graph optimizations.


- *autodiff/src/forward*: Package that implements basic forward-mode AD. Currently, this just includes a data structure named `Dual` that provides a straightforward, non-optimized AD implementation as described above. Further, we may expand upon this to include a lazy evaluation method using computational graphs.


- *autodiff/src/tests*: This package will house our testing suite for all functions and methods in our *autodiff/src/* folder. This will be used by `pytest` and TravisCI to test our project before deployment and CodeCov to report test coverage.


**Test suite location, TravisCI, and CodeCov**

Our test suites will be located in *autodiff/src/tests* and will use the `pytest` framework to easily create simple and custom tests for all of our source code. TravisCI will be used to automatically build, test, and deploy our AD package. Build status will be include on our main README.md file and we will be notified if the build fails. `pytest` and `pytest-cov` will generate code coverage reports and metrics during the TravisCI build which will be used by CodeCov to gather and report the percentage of our code base that is covered by our testing suite. A CodeCov badge with this percentage will also be displayed on our main README.


**Packaging and distribution**

Our AD package will be built using a `PEP 517` package builder, `build`, developed by PyPA (e.g. `python3 -m build .`). This will create our source distribution and wheels. This is a simple and straightforward package builder without too much overhead for our purposes. Our AD package will be uploaded to TestPyPI and PyPI using `twine`. We also include source distribution and wheels under Releases in our GitHub repository.

## Part 5. Implementation

Discuss how you plan on implementing the forward mode of automatic differentiation.
> Core Data Structures**
 - Dual objects
 - numpy array 
 - float
    

> Implemented Classes**

 - We'll need to implement Dual class for forward mode. 
 
> Method and Attributes of the Classes**

```
Dual Class
  |
  |
  Attributes ───
  |           | ─── val: float
  |           └ ─── der: length m vector where m is the dimension of independent variable
  |            
  Methods    ───
              | ─── Memeber Method: __init__(self,val,der): create a Dual object
              |                     __add__(self,other):performs addition on current dual number
              |                     __mul__(self,other):performs multiplication on current dual number
              |                     __rmul__(self, other): performs multiplication from the other direction
              |                     __sub__(self,other):performs subtraction on current dual number
              |                     __truediv__(self,other):performs division on current dual number
              |                     __rtruediv__(self,other):division from the other direction
              |                     __str__(self): return string representation of current dual number
              |                     _compatible(self, other, operand=None):Check if other element is of correct type
              |                     ...
              |                     
              |
              └ ─── Static Method:  fromArray(x):returns a list of dual numbers created from user input x
                                    constant(val, ndim=1): Create a Dual number representing a constant
              

```  

> External Dependencies**

 - We plan on using numpy. 

> deal with elementary functions like sin, sqrt, log, and exp (and all the others)**

 - We plan on create a module called Operations.py under the autodiff/src/forward directory. We'll define elementary functions such as sin,sqrt,log and exp in this file. When a user needs to do any operation, user will import the module and use the function defined in Operations.py. Each of the elementary function accepts integer,floats or dual number in its parameter and returns a new dual number.

```
Operation Module
  |
  |
  Methods  ───
            | ─── sin(x): returns a new Dual number after sin operation
            | ─── cos(x): returns a new Dual number after cos operation
            | ─── log(x,base=e): returns a new Dual number after log operation with specified base
            | ─── exp(x): returns a new Dual number after exp operations
           ... 
            

```



## Part6. Licensing

> - Helper to choose a license
> - Licenses
> - License recommendations
> - License compatibility
> - Extensive list of open source licenses

- We decide to use BSD 3-Clause “New” or “Revised” License. The only external dependency that we use is Numpy. Numpy is copyright under BSD 3-Clause "New" or "Revised" License. Under this license, redistribution and use is permitted if we retain the same copyright notice. So, we don't need to deal with patents. In addition, we want others to be able to use and redistribute our code as well. By using the same license, we pass along the same amount of freedom to use and distribute. 

```txt
BSD 3-Clause License

Copyright (c) [year], [fullname]
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```

##  Part 7. Milestone 1 Comments and Feedback

 > Good work! Really enjoyed your "How to Use" section, very detailed! Having a computational graph and an evaluation trace in the "Background" will be helpful in providing users of this package some more general information and a visualization of how AD works.

Modification:

- Modified and polished the background part.
  
- Added computational graph and evaluation trace.